# 1 第一个爬虫

In [1]:
#！/usr/bin/python
# coding: UTF-8

import requests
from bs4 import BeautifulSoup
import pandas as pd
from pandas import Series, DataFrame

**抓取**

In [2]:
link = "http://www.santostang.com/"
# header伪装成浏览器
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36"}
r = requests.get(link, headers=headers)  # respond

**解析**

In [3]:
soup = BeautifulSoup(r.text, "lxml")  # 使用BeautifulSoup解析这段文本
title = soup.find("h1", class_="post-title").a.text.strip()  # class为“post-title”的<h1>标签下的<a>
title

'第四章 – 4.3 通过selenium 模拟浏览器抓取'

**存储**

In [ ]:
path = "./files/"
ser = Series(data=[title])
ser.to_csv(path+"title.csv",sep=",", header=True)
pd.read_csv(path+"title.csv", sep=",")

,Unnamed: 0,0
0,0,第四章 – 4.3 通过selenium 模拟浏览器抓取


In [ ]:
with open(path+"/title.txt", "a+") as f:
    f.write(title)
    f.close()

# 2 静态网页抓取

## 2.1 获取响应内容

In [ ]:
import requests

In [4]:
link = "http://www.santostang.com/"
# header伪装成浏览器
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36"}
r = requests.get(link, headers=headers)  # respond

In [5]:
print("文本编码：", r.encoding)
print("响应码状态：", r.status_code)
print("相应体：", r.text)

文本编码： None
响应码状态： 503
相应体： Unable to round-trip http request to upstream: EOF


**r.tetx:**服务器响应内容，会自动根据头部字符编码进行解码  
**r.encoding:**服务器内容使用的文本编码  
**r.status_code:**
1. 200:请求成功
2. 4xx:客户端错误
3. 5xx:服务端错误

**r.content:**字节方式的响应体，会自动解码gzip和deflate编码的相应数据    
**r.json:**Requests中内置的JSON解码器

## 2.2 定制Requests

### 2.2.1 传递URL参数

In [ ]:
import requests

In [ ]:
link = "https://httpbin.org/get"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"}
key_dict = {"key1": "value1", "key2": "value2"}
r = requests.get(link, headers=headers, params=key_dict)
print("URL:", r.url)
print("字符串响应体:\n", r.text)

### 2.2.2 定制请求头

In [ ]:
import requests

In [8]:
link = "http://www.santostang.com/"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)",
    "Host": "www.santostang.com"
}
r = requests.get(link, headers=headers, timeout=5)  # timeout：超时设置
r = requests.get(link, headers=headers)

### 2.2.3 发送POST请求

In [ ]:
import requests

In [13]:
link = "https://httpbin.org/post"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"}
key_dict = {"key1": "value1", "key2": "value2"}
r = requests.post(link, data=key_dict)

In [14]:
print(r.text)

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "key1": "value1", 
    "key2": "value2"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "23", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.21.0"
  }, 
  "json": null, 
  "origin": "223.214.149.220, 223.214.149.220", 
  "url": "https://httpbin.org/post"
}



**总结**  
GET：以关键字的形式搜索  
POST：用表单传递数据

## 2.3 TOP250电影数据

In [139]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pandas import Series, DataFrame
import time
import re

**抓取数据**

In [34]:
addr = "https://movie.douban.com/top250?start=%s&filter="
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36"}
movies = list()

for i in range(0, 250, 25):
    r = requests.get(addr, headers=headers, timeout=10)
    movies.append(r)
    print("网页%s状态响应码：" % i, r.status_code)
    time.sleep(0.5)  # 暂停0.5s

网页0状态响应码： 200
网页25状态响应码： 200
网页50状态响应码： 200
网页75状态响应码： 200
网页100状态响应码： 200
网页125状态响应码： 200
网页150状态响应码： 200
网页175状态响应码： 200
网页200状态响应码： 200
网页225状态响应码： 200


**解析数据**

In [220]:
df = DataFrame(columns=["Rank", "Title", "Director", "Actor", "Year", "Area", "Genres", 
                        "Rating", "Comment_Num", "Quote", "Link_pic", "Link_detail", "Status"])
ranks = Series()
titles = Series()
directors = Series()
actors = Series()
years = Series()
areas = Series()
genres = Series()
ratings = Series()
comment_nums = Series()
quotes = Series()
pics = Series()
details = Series()
statuses = Series()

In [278]:
# 序号
soup = BeautifulSoup(movies[0].text, "lxml")
rank_list = soup.find_all("div", attrs={'class':'pic'})
# for each in rank_list:
#     print(each.em.text.strip())

In [280]:
# 标题
soup = BeautifulSoup(movies[0].text, "lxml")
title_list = soup.find_all("div", attrs={'class':'hd'})
# for each in title_list:
#     print(each.a.span.text.strip())

In [281]:
# 信息
soup = BeautifulSoup(movies[0].text, "lxml")
info_list = soup.find_all("div", attrs={'class':'bd'})


def get_info(info):
    split_info = info.p.text.strip().split("\n")
    if len(split_info) == 1:
        return
    da, yag = split_info
    director, actor = da.split("\xa0\xa0\xa0")
    director = director.split(": ")[1]
    if len
    actor = actor.split(": ")[1]
    year, area, genres = yag.strip().split("\xa0/\xa0")
    
    result = {
        "Director": director,
        "Actor": actor,
        "Year": year,
        "Area": area,
        "Genres": genres
    }
    return result

num = 0
for each in info_list:
    print(num, get_info(each))
    num += 1

0 None
1 {'Director': '弗兰克·德拉邦特 Frank Darabont', 'Actor': '蒂姆·罗宾斯 Tim Robbins /...', 'Year': '1994', 'Area': '美国', 'Genres': '犯罪 剧情'}
2 {'Director': '陈凯歌 Kaige Chen', 'Actor': '张国荣 Leslie Cheung / 张丰毅 Fengyi Zha...', 'Year': '1993', 'Area': '中国大陆 香港', 'Genres': '剧情 爱情 同性'}
3 {'Director': '吕克·贝松 Luc Besson', 'Actor': '让·雷诺 Jean Reno / 娜塔莉·波特曼 ...', 'Year': '1994', 'Area': '法国', 'Genres': '剧情 动作 犯罪'}
4 {'Director': '罗伯特·泽米吉斯 Robert Zemeckis', 'Actor': '汤姆·汉克斯 Tom Hanks / ...', 'Year': '1994', 'Area': '美国', 'Genres': '剧情 爱情'}
5 {'Director': '罗伯托·贝尼尼 Roberto Benigni', 'Actor': '罗伯托·贝尼尼 Roberto Beni...', 'Year': '1997', 'Area': '意大利', 'Genres': '剧情 喜剧 爱情 战争'}
6 {'Director': '詹姆斯·卡梅隆 James Cameron', 'Actor': '莱昂纳多·迪卡普里奥 Leonardo...', 'Year': '1997', 'Area': '美国', 'Genres': '剧情 爱情 灾难'}
7 {'Director': '宫崎骏 Hayao Miyazaki', 'Actor': '柊瑠美 Rumi Hîragi / 入野自由 Miy...', 'Year': '2001', 'Area': '日本', 'Genres': '剧情 动画 奇幻'}
8 {'Director': '史蒂文·斯皮尔伯格 Steven Spielberg', 'Actor': '连姆·尼森 Liam Neeson...', '

IndexError: list index out of range

In [235]:
# 评分
soup = BeautifulSoup(movies[0].text, "lxml")
rating_list = soup.find_all("span", attrs={'class':'rating_num'})

# for each in rating_list:
#     print(each.text.strip())

In [237]:
# quote
soup = BeautifulSoup(movies[0].text, "lxml")
quote_list = soup.find_all("span", attrs={'class':'inq'})

# for each in quote_list:
#     print(each.text.strip())

In [246]:
# detail
soup = BeautifulSoup(movies[0].text, "lxml")
detail_list = soup.find_all("div", attrs={'class':'pic'})

# for each in detail_list:
#     print(each.a["href"])

In [253]:
# pic
soup = BeautifulSoup(movies[0].text, "lxml")
pic_list = soup.find_all("div", attrs={'class':'pic'})

# for each in pic_list:
#     print(each.a.img["src"])

In [256]:
# 状态码
soup = BeautifulSoup(movies[0].text, "lxml")
status_list = soup.find_all("span", attrs={'class':'playable'})

# for each in status_list:
#     print(each.text.strip())

In [260]:
def get_data(movies):
    soup = BeautifulSoup(movies.text, "lxml")
    
    directors = Series()
    actors = Series()
    years = Series()
    areas = Series()
    genres = Series()
    ratings = Series()
    comment_nums = Series()
    quotes = Series()
    pics = Series()
    details = Series()
    statuses = Series()
    result = DataFrame()
    
    # 排名
    ranks = Series()
    rank_list = soup.find_all("div", attrs={'class':'pic'})
    for each in rank_list:
        ranks.append(each.em.text.strip())
    result["Ranks"] = ranks
    
    # 标题
    titles = Series()
    title_list = soup.find_all("div", attrs={'class':'hd'})
    for each in title_list:
        titles.append(each.a.span.text.strip())
    result["Titles"] = titles
    
    # 
    
    
    

In [266]:
df = DataFrame(data = [[1, 2, 3]], columns=list("abc"))
df.append(df)

,a,b,c
0,1,2,3
0,1,2,3
